In [1]:
import os
import json
import cv2
import albumentations as A
from tqdm import tqdm

C:\Users\Oreki\AppData\Local\Programs\Python\Python312\Lib\site-packages\albumentations\__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.4' (you have '2.0.3'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
input_images_dir = "VS_raw"
input_annotation_file = "VS_raw/VS_raw.json"
output_images_dir = "VS_V"
output_annotation_file = "VS_V/VS_V.json"
os.makedirs(output_images_dir, exist_ok=True)

with open(input_annotation_file, "r") as f:
    coco_data = json.load(f)

# Albumentations Augmentation Pipeline
transform = A.Compose(
    [
        # Rotate images slightly and fit within the original frame by scaling down. 
        A.SafeRotate(limit=(-4, 4), fill=255, p=1.0), 
        
        # Flip the image vertically around the x-axis. 
        A.VerticalFlip(p=1.0), 
    ],
    bbox_params=A.BboxParams(format="coco", label_fields=["category_ids"]), 
    keypoint_params=A.KeypointParams(format='xy', remove_invisible=False)  # Handle coco segmentation
)


# Process each image and its annotations
augmented_images = []
augmented_annotations = {"images": [], "annotations": [], "categories": coco_data["categories"]}
annotation_id = 1


for image_info in tqdm(coco_data["images"], desc="Processing Images"):
    image_id = image_info["id"]
    file_name = image_info["file_name"]
    image_path = os.path.join(input_images_dir, file_name)

    # Load the image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Get annotations related to this image
    annotations = [ann for ann in coco_data["annotations"] if ann["image_id"] == image_id]
    bboxes = [ann["bbox"] for ann in annotations]
    category_ids = [ann["category_id"] for ann in annotations]

    # Process segmentation (convert polygons to keypoints)
    segmentations = [ann.get("segmentation", []) for ann in annotations]
    keypoints_list = []
    for seg in segmentations:
        keypoints = []
        if seg:
            # Handle both single and multiple polygons
            if isinstance(seg[0], list):  # Multiple polygons
                for poly in seg:
                    if len(poly) % 2 == 0:
                        keypoints.extend([(poly[i], poly[i + 1]) for i in range(0, len(poly), 2)])
                    else:
                        print(f"Warning: Incomplete polygon detected: {poly}")
            else:  # Single polygon
                if len(seg) % 2 == 0:
                    keypoints = [(seg[i], seg[i + 1]) for i in range(0, len(seg), 2)]
                else:
                    print(f"Warning: Incomplete segmentation detected: {seg}")
        keypoints_list.append(keypoints)

    # Apply augmentation
    augmented = transform(
        image=image,
        bboxes=bboxes,
        category_ids=category_ids,
        keypoints=[kp for sublist in keypoints_list for kp in sublist]  # Flatten the keypoints list
    )

    augmented_image = augmented["image"]
    augmented_bboxes = augmented["bboxes"]
    augmented_category_ids = augmented["category_ids"]
    augmented_keypoints = augmented["keypoints"]

    # Reshape keypoints back into segmentation format
    index = 0
    new_segmentations = []
    for keypoints in keypoints_list:
        num_points = len(keypoints)
        if num_points > 0:
            segment = [round(coord, 3) for point in augmented["keypoints"][index:index + num_points] for coord in point]
            new_segmentations.append(segment)  # Correct format: [ [x1, y1, x2, y2, ...] ]
            index += num_points
        else:
            new_segmentations.append([])

    # Save the augmented image
    new_file_name = f"{os.path.splitext(file_name)[0]}_V.png"  # Remember to change the suffix
    cv2.imwrite(os.path.join(output_images_dir, new_file_name), cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR))

    # Update COCO annotations
    new_image_id = image_id  # Don't have to make new id. Duplicate ids will be resolved in Merge Annotations. 
    augmented_images.append({
        "id": new_image_id,
        "file_name": new_file_name,
        "height": augmented_image.shape[0],
        "width": augmented_image.shape[1],
    })

    for bbox, category_id, segmentation in zip(augmented_bboxes, augmented_category_ids, new_segmentations):
        bbox = [round(coord, 3) for coord in bbox]
        area = round(bbox[2] * bbox[3], 3)
        category_id = int(category_id)

        augmented_annotations["annotations"].append({
            "id": annotation_id,
            "image_id": new_image_id,
            "category_id": category_id,
            "segmentation": [segmentation] if segmentation else [],
            "area": area,
            "bbox": bbox,
            "iscrowd": 0,
        })
        annotation_id += 1



# Save the updated annotations
augmented_annotations["images"] = augmented_images
with open(output_annotation_file, "w") as f:
    json.dump(augmented_annotations, f, indent=4)

print("Augmentation complete!")

Processing Images: 100%|█████████████████████████████████████████████████████████████████| 9/9 [00:03<00:00,  2.72it/s]

Augmentation complete!
